In [1]:
import numpy as np
import pywt
class time_feats:
    def __init__(self):
        pass
    @staticmethod
    def compositionAcc(X):
        return np.array(np.sqrt((X*X).dot(np.ones((X.shape[1], 1))))).flatten()
    @staticmethod
    def std(X):
        return np.std(X, axis = 0)
    @staticmethod
    def mean(X):
        return np.mean(X, axis = 0)
    @staticmethod
    def max(X):
        return np.max(X, axis = 0)
    @staticmethod
    def min(X):
        return np.min(X, axis = 0)
    @staticmethod
    def range(X):
        return np.max(X, axis = 0) - np.min(X, axis = 0)
    @staticmethod
    def greaterThanMeanNum(X):
        return np.ones((1, X.shape[0])).dot(X > np.mean(X, axis = 0))
    @staticmethod
    def cos_sim(X):
        xx = sum([i**2 for i in X[:, 0]])
        yy = sum([j**2 for j in X[:, 1]])
        zz = sum([k**2 for k in X[:, 2]])
        xy = sum([i*j for i, j in zip(X[:, 0], X[:, 1])])
        xz = sum([i*k for i, k in zip(X[:, 0], X[:, 2])])
        yz = sum([j*k for j, k in zip(X[:, 1], X[:, 2])])
        cos_sim_xy = xy / np.sqrt(xx*yy)
        cos_sim_xz = xz / np.sqrt(xx*zz)
        cos_sim_yz = yz / np.sqrt(yy*zz)
        return [cos_sim_xy, cos_sim_xz, cos_sim_yz]
    @staticmethod
    def rms(X):
        return np.sqrt(sum([i**2 for i in X]) / len(X))
    @staticmethod
    def average_abs(X):
        return sum([np.abs(i) for i in X]) / len(X)
    
        

class freq_feats:
    def __init__(self):
        pass
    @staticmethod
    def wpd_energy(signal, n):
        # 小波包分解各级能量系数
        # 使用合成加速度
        assert signal.ndim == 1 
        map = {}
        map[1] = signal
    
        wp = pywt.WaveletPacket(data=signal, wavelet='db8',mode='symmetric',maxlevel=n)
        lev = []
        path = []
        for i in [node.path for node in wp.get_level(n, 'freq')]:
            path.append(i)
            map[i] = wp[i].data
        energy = []
        for i in range(1,pow(2, n)+1):
            energy.append(pow(np.linalg.norm(map[path[i-1]], ord=2), 2))
        return energy
    @staticmethod
    def wpd_rec_plt(signal, n):
        #计算每一个节点的系数，存在map中，key为'aa'等，value为列表
        assert signal.ndim == 1
        map = {}
        map[1] = signal
        #作图
        plt.figure(figsize=(15, 25))
        plt.subplot(pow(2, n)+1,1,1) #绘制第一个图
        plt.title('Raw signal')
        plt.plot(map[1])
        #wpd分解
        wp = pywt.WaveletPacket(data=signal, wavelet='db8',mode='symmetric',maxlevel=n)
        for i in [node.path for node in wp.get_level(n, 'freq')]:
            map[i] = wp[i].data
            # 打印系数
            print(wp[i].data)
        path = [node.path for node in wp.get_level(n, 'freq')]
        resolution = len(signal)//2//pow(2, n)
        for i in range(2,pow(2, n)+2):
            new_wp = pywt.WaveletPacket(data=None, wavelet='db8',mode='symmetric',maxlevel=n)
            new_wp[path[i-2]] = map[path[i-2]]
            plt.subplot(pow(2, n)+1, 1, i)
            plt.title('{}Hz—{}Hz'.format((i-2)*resolution, (i-1)*resolution))
            plt.plot(new_wp.reconstruct(update = False)) #列表从0开始
        new_wp = pywt.WaveletPacket(data=None, wavelet='db4',mode='symmetric',maxlevel=n)
        for i in path:
            new_wp[i] = map[i]
            
    @staticmethod
    def wpd_denoise_plt(signal, n):
        #计算每一个节点的系数，存在map中，key为'aa'等，value为列表
        assert signal.ndim == 1
        map = {}
        map[1] = signal
        #作图
        plt.figure(figsize=(15, 25))
        plt.subplot(pow(2, n)+1,1,1) #绘制第一个图
        plt.title('Raw signal')
        plt.plot(map[1])
        #wpd分解
        wp = pywt.WaveletPacket(data=signal, wavelet='db8',mode='symmetric',maxlevel=n)
        for i in [node.path for node in wp.get_level(n, 'freq')][1:]:# 最低频不动
            map[i] = wp[i].data
            wp[i].data = pywt.threshold(wp[i].data, 0.8*max(wp[i].data)) # 将阈值以下系数全部置零
            print(wp[i].data)
        plt.subplot(pow(2, n)+1,1,2)
        plt.plot(wp.reconstruct(update = False))
    @staticmethod
    def wpd_coeffs_plt(signal,n):
        # 打印系数
        wp = pywt.WaveletPacket(data=signal, wavelet='db8',mode='symmetric',maxlevel=n)

        #计算每一个节点的系数，存在map中，key为'aa'等，value为列表
        map = {}
        map[1] = signal
        for row in range(1,n+1):
            lev = []
            for i in [node.path for node in wp.get_level(row, 'freq')]:
                map[i] = wp[i].data

        #作图
        plt.figure(figsize=(15, 10))
        plt.subplot(n+1,1,1) #绘制第一个图
        plt.plot(map[1])
        for i in range(2,n+2):
            level_num = pow(2,i-1)  #从第二行图开始，计算上一行图的2的幂次方
            #获取每一层分解的node：比如第三层['aaa', 'aad', 'add', 'ada', 'dda', 'ddd', 'dad', 'daa']
            re = [node.path for node in wp.get_level(i-1, 'freq')]  
            for j in range(1,level_num+1):
                plt.subplot(n+1,level_num,level_num*(i-1)+j)
                plt.plot(map[re[j-1]]) #列表从0开始
